#  Spark Streaming-Behavior-Anomaly-Detection 
We will use Streaming data to detect anomlay in real time .

We are ready to build t Machine learning program. we will proceed as follow:


Step 1) Basic operation with PySpark

Step 2) Data preprocessing

Step 3) Build a data processing pipeline

Step 4) Build the classifier

Step 5) Train and evaluate the model

Step 6) Tune the hyperparameter

# Importing Packages and configuring spark engine

In [11]:
import os
import sys
import re
import time
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark.context import SparkContext 

spark = SparkSession.builder.getOrCreate()#create spark session 
sc = spark.sparkContext#create sparkContext
from pyspark.sql.types import *
from pyspark.sql import Row
# from pyspark.sql.functions import *
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pyspark.sql.functions as func
import matplotlib.patches as mpatches
from operator import add
from operator import add
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
import itertools

# 1. Load Dataset

In [12]:
import pandas as pd 
df = pd.read_pickle("C:/Users/rzouga/Downloads/Work Swiss/location-exploration/Recruitment-Challenge/locations.pkl")
dfspark = spark.createDataFrame(df)

In [13]:
dfspark.printSchema()

root
 |-- customerLocationState: string (nullable = true)
 |-- startTime: timestamp (nullable = true)
 |-- endTime: timestamp (nullable = true)
 |-- duration: long (nullable = true)



## 1. Create new features :

In [14]:
import pyspark.sql.functions as F
from pyspark.sql import Window

from pyspark.sql import functions as F
dfspark =dfspark.withColumn("hourstart", F.hour(F.col("startTime")))
dfspark=dfspark.withColumn("date", F.to_date(F.col("startTime")))



cumSumByHour = Window.partitionBy(['date','customerLocationState','hourStart']).orderBy('startTime').rowsBetween(-sys.maxsize, 0)

dfspark = dfspark.withColumn("cumSumByHour",F.count('customerLocationState').over(cumSumByHour))
cumSumBydate= Window.partitionBy(['date','customerLocationState']).orderBy('startTime').rowsBetween(-sys.maxsize, 0)

dfspark = dfspark.withColumn("cumSumBydate",F.count('customerLocationState').over(cumSumBydate))

dfspark.select('customerLocationState',"date",'hourStart','cumSumByHour',
                               'cumSumBydate').show(10, False)

+---------------------+----------+---------+------------+------------+
|customerLocationState|date      |hourStart|cumSumByHour|cumSumBydate|
+---------------------+----------+---------+------------+------------+
|outside              |2018-06-26|8        |1           |1           |
|outside              |2018-07-09|8        |1           |1           |
|unknown              |2018-07-16|11       |1           |1           |
|unknown              |2018-07-16|15       |1           |2           |
|bedroom              |2018-07-21|3        |1           |1           |
|bedroom              |2018-07-21|3        |2           |2           |
|bedroom              |2018-07-21|7        |1           |3           |
|bedroom              |2018-07-21|10       |1           |4           |
|bedroom              |2018-07-21|12       |1           |5           |
|bedroom              |2018-07-21|15       |1           |6           |
+---------------------+----------+---------+------------+------------+
only s

In [15]:
dfsparkML=dfspark.select('customerLocationState','hourStart','cumSumByHour',
                               'cumSumBydate')
dfsparkML.printSchema()

root
 |-- customerLocationState: string (nullable = true)
 |-- hourStart: integer (nullable = true)
 |-- cumSumByHour: long (nullable = false)
 |-- cumSumBydate: long (nullable = false)



# 2. Data Preparation
## Feature Engineering in pyspark

The most commonly used data pre-processing techniques in approaches in Spark are as follows

1) VectorAssembler

2)Bucketing

3)Scaling and normalization

4) Working with categorical features

5) Text data transformers

6) Feature Manipulation

7) PCA

## Convert the data to dense vector (features):

Note: You are strongly encouraged to try my get_dummy function for dealing with the categorical data in complex dataset.


In [16]:
def get_dummy(df,categoricalCols,continuousCols):
    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder,VectorAssembler
    from pyspark.sql.functions import col
    indexers = [ StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c)) for c in categoricalCols ]
     # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),outputCol="{0}_encoded".format(indexer.getOutputCol())) for indexer in indexers ]
    
    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]+ continuousCols, outputCol="features")
    
    pipeline = Pipeline(stages=indexers + encoders + [assembler])
    model=pipeline.fit(df)
    data = model.transform(df)
    #data = data.withColumn('label',col(labelCol))
    return data.select('features')

catcols = ['customerLocationState']
num_cols = ['hourStart','cumSumByHour','cumSumBydate']
#labelCol ='label'
data = get_dummy(dfsparkML,catcols,num_cols)
data.show(3)

+--------------------+
|            features|
+--------------------+
|(10,[4,7,8,9],[1....|
|(10,[4,7,8,9],[1....|
|(10,[5,7,8,9],[1....|
+--------------------+
only showing top 3 rows



In [17]:
# nubmbers of rows
n = data.count()
n

38864

## Preprocessing :
**Note:** Since clustering algorithms including k-means use distance-based measurements to determine the
similarity between data points, It’s strongly recommended to standardize the data to have a mean of zero
and a standard deviation of one.

In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer, VectorIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans
# Let us create an object of MinMaxScaler class
MinMaxScaler=MinMaxScaler().setInputCol("features").setOutputCol("Scaled_features")
scalerModel= MinMaxScaler.fit(data)
scaledData = scalerModel.transform(data)

# choose the best K

In [23]:
"""import numpy as np

cost = np.zeros(10)
for k in range(2,10):
    kmeans = KMeans()\
              .setK(k)\
              .setSeed(1) \
              .setFeaturesCol("Scaled_features")\
              .setPredictionCol("cluster")
    model = kmeans.fit(scaledData)
    cost[k] = model.computeCost(scaledData) # requires Spark 2.0 or later"""

'import numpy as np\n\ncost = np.zeros(10)\nfor k in range(2,10):\n    kmeans = KMeans()              .setK(k)              .setSeed(1)               .setFeaturesCol("Scaled_features")              .setPredictionCol("cluster")\n    model = kmeans.fit(scaledData)\n    cost[k] = model.computeCost(scaledData) # requires Spark 2.0 or later'

In [24]:
""""import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sbs
from matplotlib.ticker import MaxNLocator
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),cost[2:10])
ax.set_xlabel('k')
ax.set_ylabel('cost')
ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.show()"""

'"import numpy as np\nimport matplotlib.mlab as mlab\nimport matplotlib.pyplot as plt\nimport seaborn as sbs\nfrom matplotlib.ticker import MaxNLocator\nfig, ax = plt.subplots(1,1, figsize =(8,6))\nax.plot(range(2,10),cost[2:10])\nax.set_xlabel(\'k\')\nax.set_ylabel(\'cost\')\nax.xaxis.set_major_locator(MaxNLocator(integer=True))\nplt.show()'

 Based on  the cost , we chose K=10
###  Train the machine learning model

In [25]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(
    featuresCol="Scaled_features", 
    predictionCol="cluster", 
    k=10, 
    seed=0)

model = kmeans.fit(scaledData)


In [26]:
centroids=model.clusterCenters()
centroids[0]

array([1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.53799194, 0.10801065, 0.23640452])

### Assign clusters

In [27]:
prediction = model.transform(scaledData)

prediction.groupBy("cluster").\
	count().\
    orderBy("cluster").\
    show()
prediction.select("Scaled_features", 
                  "cluster").show(5)

+-------+-----+
|cluster|count|
+-------+-----+
|      0| 7043|
|      1| 5064|
|      2| 4395|
|      3| 5272|
|      4| 7111|
|      5| 2118|
|      6|  625|
|      7|  424|
|      8| 1858|
|      9| 4954|
+-------+-----+

+--------------------+-------+
|     Scaled_features|cluster|
+--------------------+-------+
|[0.0,0.0,0.0,0.0,...|      6|
|[0.0,0.0,0.0,0.0,...|      6|
|[0.0,0.0,0.0,0.0,...|      7|
|[0.0,0.0,0.0,0.0,...|      7|
|[0.0,0.0,0.0,1.0,...|      5|
+--------------------+-------+
only showing top 5 rows



The threshold has strong impact on the result. Be careful when choosing it! A simple way to choose the threshold's value is picking up a distance of a data point from among known data. For example, the 100th-farthest data point distance can be an option.

#### calculate the distance between each point and the centroid of the cluster it belongs to:

In [28]:
import numpy as np 
def get_distance_map(record,culster):
    #cluster = clusters.predict(record)
    centroid = model.clusterCenters()[cluster]
    dist = np.linalg.norm(record - centroid)
    return (dist)


prediction =prediction.select("Scaled_features","cluster")
def to_distances(entry):
    data_point = entry[0]
    prediction = entry[1]
    centroid = model.centers[prediction]
    return np.linalg.norm(data_point - centroid)





### Calculation of centroids for every cluster

In [29]:
train_clusters = model.clusterCenters()

traind_clusters = {int(i):[float(train_clusters[i][j]) for j in range(len(train_clusters[i]))] 
              for i in range(len(train_clusters))}
train_clusters

[array([1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.53799194, 0.10801065, 0.23640452]),
 array([0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.79982313, 0.05952381, 0.22949373]),
 array([0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.56512836, 0.02203803, 0.07130262]),
 array([0.00000000e+00, 9.99810319e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.89681335e-04, 3.98132876e-01,
        5.15662259e-02, 7.89686868e-02]),
 array([1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.8387323 , 0.10297093, 0.45606624]),
 array([0.        , 0.        , 0.        , 0.99952786, 0.        ,
        0.        , 0.        , 0.31861477, 0.01794145, 0.02942536]),
 array([0.        , 0.        , 0.        , 0.        , 0.9984    ,
        0.        , 0.        , 0.58525217, 0.00685714, 

In [30]:
train_df_centers = spark.sparkContext.parallelize([(k,)+(v,) for k,v in traind_clusters.items()]).toDF(['cluster','center'])
train_df_centers.toPandas().head()

,cluster,center
0,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53799193..."
1,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313..."
2,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.56512835..."
3,3,"[0.0, 0.9998103186646434, 0.0, 0.0, 0.0, 0.0, ..."
4,4,"[0.9999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [31]:
train_pred_df = prediction.withColumn('cluster',F.col('cluster').cast(IntegerType()))
train_pred_df.toPandas().head()

,Scaled_features,cluster
0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.34782608...",6
1,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.34782608...",6
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.47826086...",7
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.65217391...",7
4,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.13043478...",5


###  Joining of centroid and feature dataframes

In [32]:
train_pred_df = train_pred_df.join(train_df_centers,on='cluster',how='left')
train_pred_df.toPandas().head()

,cluster,Scaled_features,center
0,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.43478260...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53799193..."
1,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.43478260...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53799193..."
2,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.43478260...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53799193..."
3,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.43478260...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53799193..."
4,0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.43478260...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53799193..."


# Finding Anomaly Values
#### Getting distance values function

In [33]:
get_dist = F.udf(lambda features, center :
                 float(features.squared_distance(center)),FloatType())

### Sorting the furthest distance values

In [34]:
train_pred_df = train_pred_df.withColumn('dist',get_dist(F.col('Scaled_features'),F.col('center')))
train_pred_df.toPandas().sort_values(by="dist",ascending=False).head(10)



,cluster,Scaled_features,center,dist
11524,9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.17391304...","[0.9997964169381108, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2.018197
24323,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.43478260...","[0.0, 0.9998103186646434, 0.0, 0.0, 0.0, 0.0, ...",2.007943
7189,7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.78260869...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.9929245283018867, ...",1.990329
7188,7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.78260869...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.9929245283018867, ...",1.989536
14083,5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.39130434...","[0.0, 0.0, 0.0, 0.9995278564683664, 0.0, 0.0, ...",1.005527
8046,6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.56521739...","[0.0, 0.0, 0.0, 0.0, 0.9984000000000001, 0.0, ...",0.997304
7433,7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.65217391...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.9929245283018867, ...",0.986932
32873,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.73913043...","[0.9999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.920326
32872,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.73913043...","[0.9999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.830496
32871,4,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.73913043...","[0.9999999999999999, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.745418


# I- Fisrt method to detect Anomlay 
Calculate the number of outliers based on the given fraction and flag outliers:

number_of_outliers = int(outliers_fraction * distances.count())

distances.count()=Numbers of rows =38864

In [35]:
# let's take 0.001 as fraction 
outliers_fraction=0.001
distances_count=38864
number_of_outliers = int(outliers_fraction * distances_count)
outliers = train_pred_df.orderBy(train_pred_df["dist"].desc()).take(number_of_outliers)


In [36]:
outliers[0:3]

[Row(cluster=9, Scaled_features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.1739, 0.0, 0.0]), center=[0.9997964169381108, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00020358306188925082, 0.2728101543690696, 0.06701760508763766, 0.06884076411509232], dist=2.0181968212127686),
 Row(cluster=3, Scaled_features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.4348, 0.0, 0.0104]), center=[0.0, 0.9998103186646434, 0.0, 0.0, 0.0, 0.0, 0.00018968133535660092, 0.3981328758989246, 0.05156622588337306, 0.07896868677288828], dist=2.0079429149627686),
 Row(cluster=7, Scaled_features=DenseVector([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.7826, 0.0476, 0.0104]), center=[0.0, 0.0, 0.0, 0.0, 0.0, 0.9929245283018867, 0.0047169811320754715, 0.6712469237079574, 0.011680143755615447, 0.02260220125786163], dist=1.9903289079666138)]

# II- Calculation of Threshold Value according to distance
## Average distance for every cluster


In [37]:
averageDistance = train_pred_df.groupBy("cluster").agg(F.avg("dist").alias("avgDist"))
averageDistance.toPandas().head()

,cluster,avgDist
0,1,0.026767
1,6,0.029433
2,3,0.026675
3,5,0.030295
4,9,0.023700


 ## Maximum distance for every cluster

In [38]:
maxDistance = train_pred_df.groupBy("cluster")\
.agg(F.max("dist").alias("maxDist"))
maxDistance.toPandas().head()

,cluster,maxDist
0,1,0.408354
1,6,0.997304
2,3,2.007943
3,5,1.005527
4,9,2.018197


## Joining of predicted and threshold dataframes

In [39]:
anomalyDetection = train_pred_df.join(maxDistance, maxDistance.cluster == train_pred_df.cluster)
anomalyDetection.toPandas().head(5)

,cluster,Scaled_features,center,dist,cluster,maxDist
0,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.65217391...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.032342,1,0.408354
1,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.69565217...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.019759,1,0.408354
2,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.73913043...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.011174,1,0.408354
3,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.73913043...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.006572,1,0.408354
4,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.73913043...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.006723,1,0.408354


In [40]:
# Assigning of labels as normal or anomaly
detected_df = anomalyDetection.withColumn("detected", F.when((anomalyDetection.dist > (anomalyDetection.maxDist*0.99)), "Anomaly").otherwise("Normal"))
detected_df.toPandas().head(5)

,cluster,Scaled_features,center,dist,cluster,maxDist,detected
0,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.65217391...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.032342,1,0.408354,Normal
1,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.69565217...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.019759,1,0.408354,Normal
2,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.73913043...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.011174,1,0.408354,Normal
3,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.73913043...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.006572,1,0.408354,Normal
4,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.73913043...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.79982313...",0.006723,1,0.408354,Normal


# Streaming Process

In [125]:
from pyspark.streaming import StreamingContext

spark.conf.set("spark.sql.shuffle.partitions", "5")
# set microbatch interval as 10 seconds, this can be customized according to the project
ssc = StreamingContext(sc,10)
schema = StructType(
[
    StructField("customerLocationState", StringType(), True),
    StructField("startTime",  TimestampType(), True),
    StructField("endTime",  TimestampType(), True),
    StructField("duration", DoubleType(), True)
])
locations = spark.readStream \
.format("csv")\
.option("header", True)\
.option("sep", ",")\
.schema(schema)\
.load("C:/Users/rzouga/Downloads/Work Swiss/location-exploration/Recruitment-Challenge/locations*.csv")

locations.printSchema()

root
 |-- customerLocationState: string (nullable = true)
 |-- startTime: timestamp (nullable = true)
 |-- endTime: timestamp (nullable = true)
 |-- duration: double (nullable = true)



In [133]:
import pyspark.sql.functions as F
from pyspark.sql import Window
w = F.window('startTime', '3600 seconds')
locations = locations.withColumn("processingTime",F.current_timestamp())
locations =locations.withColumn("hourstart",F.hour(F.col("startTime")))
locations=locations.withColumn("date", F.to_date(F.col("startTime")))
locations.printSchema

<bound method DataFrame.printSchema of DataFrame[customerLocationState: string, startTime: timestamp, endTime: timestamp, duration: double, processingTime: timestamp, hourstart: int, date: date]>

In [134]:
countByCustomerPerday = locations\
.selectExpr(
"customerLocationState","duration",
"date" ,
"hourstart","startTime" )\
.withWatermark("startTime", "1 hours")\
.groupBy("date","customerLocationState").count().alias("counts1")

In [135]:
countByCustomerPerHour = locations\
.selectExpr(
"customerLocationState","duration",
 "startTime",
"date" ,
"hourstart" )\
.withWatermark("startTime", "1 hours")\
.groupBy("date","customerLocationState","hourstart").count().alias("counts2")

In [148]:
countByCustomerPerHour.isStreaming 

True

In [ ]:
query= countByCustomerPerHour.writeStream\
  .format('console')\.queryName("location_counts")\
  .outputMode("append")\
  .start()


In [ ]:
from time import sleep
for x in range(3):
    spark.sql("select * from location_counts").show(3)
    sleep(2)

In [ ]:
query.stop()

# Vectorization of streaming data

In [ ]:
locationsStream = get_dummy(locations,catcols,num_cols)
DataStream= scalerModel.transform(locationsStream)

# Prediction Process
## Prediction of Streaming Data

In [ ]:
streaming_prediction = model.transform(DataStream)

## Finding Anomaly

In [ ]:


df_pred = streaming_prediction.withColumn('prediction',F.col('prediction').cast(IntegerType()))

df_pred = df_pred.join(train_df_centers,on='prediction',how='left')

## Getting Euclidean Distance for features

In [ ]:
get_dist = funcs.udf(lambda features, center : 
                 float(features.squared_distance(center)),FloatType())

df_pred = df_pred.withColumn('dist',get_dist(funcs.col('features'),funcs.col('center')))
df_pred = df_pred.withColumnRenamed("prediction", "cluster")

## Calculation of Average and Maximum Distance for Streaming

In [ ]:
detection = maxDist_streaming.withColumn("detected", funcs.when(maxDist_streaming.dist > maxDist_streaming.maxDist, "Anomaly").otherwise("Normal"))
detection = detection.select( "prediction", "dist", "maxDist", "status", "detected")

## Adding Sliding Window Time using Current Timestamp

In [ ]:
currentTimeDf = detection.withColumn("processingTime2",funcs.current_timestamp())

In [ ]:
windowedCount = currentTimeDf\
.withWatermark("processingTime", "5 seconds")\
.groupBy(F.window("processingTime", "4 seconds", "2 seconds"), "prediction", "detected", "dist", "maxDist")\
.avg("dist").orderBy("window")

# Start Streaming
## Option 1 - Using Sliding Window and Watermarking (Confusion Matrix)


In [ ]:
confusion_matrix = detection.groupBy("detected").count().select( "detected","count")
confusion_matrix = currentTimeDf\
.withWatermark("processingTime", "5 seconds")\
.groupBy(funcs.window("processingTime", "3 seconds", "1 seconds"),"status", "detected")\
.count().orderBy("window")
q = confusion_matrix.writeStream\
.outputMode("complete")\
.format("console")\
.option("truncate", "false")\
.start()
q.awaitTermination()

## Option 2 - Using Append method

In [ ]:
q = windowedCount.writeStream\
.outputMode("complete")\
.format("console")\
.option("truncate", "false")\
.start()


q.awaitTermination()



## Option 3 - Using Complete and Aggregate Function

In [ ]:
group = df_pred.groupBy(["cluster","status"]).count()\
.orderBy("cluster", "status", ascending=True)

q = group.writeStream\
.outputMode("complete")\
.format("console")\
.start()



q.awaitTermination()

